In [ ]:
import sys
sys.path.append("../src")

# Introduction to sensAI: Supervised Learning with VectorModels

In [ ]:
import sensai
import numpy as np

## Logging

sensAI will log relevant activies and inform about ongoing processes as well as results via the log. It is therefore highly recommended that logging be enabled when using sensAI.

sensAI provides a `logging` module which includes Python's standard logging module and adds some additional functionality. To enable logging, simply use its `configureLogging` function.


In [ ]:
from sensai.util import logging

logging.configureLogging(level=logging.INFO)

To additionally write log output to a file, use the function `logging.addFileLogger`.

## VectorModels

The central base class for supervised learning problems in sensAI is `VectorModel`. A VectorModel is any model which operates on data points that can be reprsented as vectors of data. Here, vector is to be understood not in the mathematical sense but in the computer science sense, where a vector is simply an array of (potentially arbitaririly complex) data. (The mathematical equivalent is a tuple.) Models are typically expected to be able to process more than one data point at a time and thus should be able to process a sequence of vectors. 

We use pandas DataFrames to represent such sequences of data points. Note that pandas DataFrames are not limited to primitive data types but can hold arbitrary objects in every cell. When dealing with a large number of inputs, DataFrames also provide at least limited meta-information in the form of column names, so we do not lose track of what is contained in which element of a data point (vector).

VectorModel itself is an abstract base class, which provides a lot of useful functionality that all its specialisations inherit (as we will see later, particularly in the more advanced tutorials). The class is specialised in `VectorClassificationModel` and `VectorRegressionModel`, which in turn are specialised for various machine learning frameworks (such as sklearn and PyTorch) or can be directly subclassed to create your own model. 

In this tutorial, we will be dealing with a classification problem. Therefore, we will apply subclasses of `VectorClassificationModel` such as `SkLearnRandomForestVectorClassificationModel`. As an sklearn classification model which uses a well-defined training and inference interface, the implementation of the class is essentially justa few lines of code (given the intermediate abstraction `AbstractSkLearnVectorClassificationModel` for all classification models that use the sklearn protocol).

## Training and Evaluating Models

First, let us load a dataset which we can experiment. sklearn provides, for example, the Iris classification dataset, where the task is to differentiate three different types of flowers based on measurements of their petals and sepals.

In [ ]:
import sklearn.datasets
import pandas as pd

irisData = sklearn.datasets.load_iris()
irisInputDF = pd.DataFrame(irisData["data"], columns=irisData["feature_names"]).reset_index(drop=True)
irisOutputDF = pd.DataFrame({"class": [irisData["target_names"][idx] for idx in irisData["target"]]}).reset_index(drop=True)

Here's a sample of the data, combining both the inputs and outputs:

In [ ]:
irisCombinedDF = pd.concat((irisInputDF, irisOutputDF), axis=1)
irisCombinedDF.sample(10)

When working with sensAI, we typically use DataFrames such as this as the starting point.

We create an instance of **InputOutputData** from the two data frames.

In [ ]:
irisInputOutputData = sensai.InputOutputData(irisInputDF, irisOutputDF)

### Low-Level Training and Inference 

We use a **DataSplitter** (see subclasses) to split the data into a training and test set, specifically a **DataSplitterFractional**.

In [ ]:
dataSplitter = sensai.data.DataSplitterFractional(0.8, shuffle=True)
trainingIoData, testIoData = dataSplitter.split(irisInputOutputData)

Now we are ready to train a model. Let us train a random forest classifier, which should work well for this sort of problem. sensAI provides models from various libraries, including scikit-learn, PyTorch, lightgbm, xgboost, catboost, and TensorFlow.

In this case, let us use the random forest implementation from sklearn, which is provided via the wrapper class SkLearnRandomForestVectorClassificationModel.

sensAI's **VectorModel** classes (specialised for classification and regression) provide a common interface with a lot of useful functionality, which we will see later.

In [ ]:
randomForestModel = sensai.sklearn.classification.SkLearnRandomForestVectorClassificationModel(
    min_samples_leaf=2).withName("RandomForest")

The class suppports all the parameters supported by the original sklearn model. In this case, we only set the minimum number of samples that must end up in each leaf.

We train the model using the `fitInputOutputData` method; we could also use the `fit` method, which is analogous to the sklearn interface and takes two arguments (input, output).

In [ ]:
randomForestModel.fitInputOutputData(trainingIoData)
randomForestModel

We can now apply the trained model and predict the outputs for the test set we reserved.

In [ ]:
predictedOutputsDF = randomForestModel.predict(testIoData.inputs)
predictedOutputsDF.head(5)

Let's compare some of the predictions to the ground truth.

In [ ]:
pd.concat((predictedOutputsDF.rename(columns={"class": "predictedClass"}), testIoData.outputs), axis=1).sample(10)

Using the ground truth and predicted values, we could now compute the metrics we're interested in. We could, for example, use the metrics implemented in sklearn to analyse the result. Yet sensAI already provides abstractions that facilitate the generation of metrics and the collection of results. Read on!

### Using Evaluators

sensAI provides evaluator abstractions which facilitate the training and evaluation of models.

For a classification problem, we instantiate a VectorClassificationModelEvaluator. An evaluator serves to evaluate one or more models based on the same data, so we construct it with the data and instructions on how to handle/split the data for evaluation.

In [ ]:
evaluatorParams = sensai.evaluation.VectorClassificationModelEvaluatorParams(dataSplitter=dataSplitter, computeProbabilities=True)
evaluator = sensai.evaluation.VectorClassificationModelEvaluator(irisInputOutputData, params=evaluatorParams)

We can use this evaluator to evaluate one or more models. Let us evaluate the random forest model from above.

In [ ]:
evaluator.fitModel(randomForestModel)
evalData = evaluator.evalModel(randomForestModel)

The evaluation data holds, in particular, an **EvalStats** object, which can provide data on the quality of the results.
Depending on the type of problem, many metrics will already be computed by default.

In [ ]:
evalStats = evalData.getEvalStats()
evalStats

We can get the metrics in a dictionary as follows:

In [ ]:
evalStats.metricsDict()

We can compute additional metrics by passing a metric to the `computeMetricValue` method, but we could also have added additional metrics to the `evaluatorParams` above and have the metric included in all results.

Let's see how frequently the true class is among the top two most probable classes.

In [ ]:
evalStats.computeMetricValue(sensai.eval_stats_classification.ClassificationMetricTopNAccuracy(2))

The EvalStats object can also be used to generate plots, such as a confusion matrix or a precision-recall plot for binary classification.

In [ ]:
evalStats.plotConfusionMatrix(normalize=True);

### Using the Fully-Integrated Evaluation Utilities

sensAI's evaluation utilities take things one step further and assist you in out all the evaluation steps and results computations in a single call.

You can perform evaluations based on a single split or cross-validation. We simply declare the necessary parameters for both types of computations (or the one type we seek to carry out).

In [ ]:
evaluatorParams = sensai.evaluation.VectorClassificationModelEvaluatorParams(
    dataSplitter=dataSplitter, computeProbabilities=True, 
    additionalMetrics=[sensai.eval_stats_classification.ClassificationMetricTopNAccuracy(2)])
crossValidatorParams = sensai.evaluation.crossval.VectorModelCrossValidatorParams(folds=10, 
    evaluatorParams=evaluatorParams)
evalUtil = sensai.evaluation.ClassificationEvaluationUtil(irisInputOutputData, 
    evaluatorParams=evaluatorParams, crossValidatorParams=crossValidatorParams)

In practice, we will usually want to save evaluation results. The evaluation methods of `evalUtil` take a parameter `resultWriter` which allows us to define where results shall be written. Within this notebook, we shall simply inspect the resulting metrics in the log that is printed, and we shall configure plots to be shown directly.

#### Simple Evaluation

We can perform the same evaluation as above (which uses a single split) like so:

In [ ]:
evalUtil.performSimpleEvaluation(randomForestModel, showPlots=True)

#### Customising the Set of Plots

If we decide that we don't really want to have the normalised confusion matrix, we can disable it for any further experiments.

In [ ]:
evalUtil.evalStatsPlotCollector.getEnabledPlots()

Some of these are only active for binary classification. The one we don't want is "confusion-matrix-rel".

In [ ]:
evalUtil.evalStatsPlotCollector.disablePlots("confusion-matrix-rel")

We could also define our own plot class (by creating a new subclass of `ClassificationEvalStatsPlot`) and add it to the `evalStatsPlotCollector` in order to have the plot auto-generated whenever we apply one of `evalUtil`'s methods.

#### Cross-Validation

We can similarly run cross-validation and produce the respective evaluation metrics with a single call.

In [ ]:
evalUtil.performCrossValidation(randomForestModel, showPlots=True)

As you can see, the plot we disabled earlier is no longer being generated.

#### Comparing Models

A most common use case is to compare the performance of several models. The evaluation utility makes it very simple to compare any number of models.

Let's say we want to compare the random forest we have been using thus far to a simple decision tree.

In [ ]:
results = evalUtil.compareModels([
        randomForestModel, 
        sensai.sklearn.classification.SkLearnDecisionTreeVectorClassificationModel(min_samples_leaf=2).withName("DecisionTree")], 
    useCrossValidation=True)

In addition to the data frame with the aggregated metrics, which was already printed to the log, the results object contains all the data that was generated during the evaluation. We can, for example, use it to plot the distribution of one of the metrics across all the folds for one of our models.

In [ ]:
display(results.resultsDF)

escRandomForest = results.resultByModelName["RandomForest"].crossValData.getEvalStatsCollection()
escRandomForest.plotDistribution("accuracy", bins=np.linspace(0,1,21), stat="count", kde=False);


We can also compute additional aggregations or inspect the full list of metrics.

In [ ]:
escRandomForest.aggMetricsDict(aggFns=[np.max, np.min])

In [ ]:
escRandomForest.getValues("accuracy")

## Feature Generators and Data Frame Transformers

When dealing with the preparation of input data for a model, we often need to cater to technical requirements of various types of models. sensAI seeks to make the process of supporting multiple input pipelines for different types of models as simple as possible - by focusing on concise, declarative semantics and integrating the model-specific data extraction and transformation mechanisms into the models themselves.
In essence, this means:

  1. **Starting with the raw or most general representation of the data**
 
     This could mean simply starting with the data that is straightforward for us to obtain - or using directly using particular domain specific objects.
     
     For example, if the problem is to classify situations, we might already have a ``Situation`` class in our code which represents all the data that is is relevant to a situation (e.g. the point in time, the affected user, the location, etc.). Pandas DataFrames can represent arbitrary data, so there is no reason to not simply use as the raw input data frame that is fed to our models a single column containing instances of class ``Situation``.
     Or we might instead directly observe a set of sensor readings, all of which are real numbers; this scenario would certainly be closer to what we typically see in machine learning data sets, but it isn't always the case in the real world.
     
     Whatever the case may be, we can represent it in a data frame. We call the original input data frame, which we pass to a sensAI ``VectorModel``, the *raw data frame*.

  2. **Extracing features from the raw data, using their "natural" representation** (using ``FeatureGenerators``)
     
     We extract from the raw data frame pieces of information that we regard as relevant *features* for the task at hand.
     A sensAI ``FeatureGenerator`` can generate one or more data frame columns (containing arbitrary data), and a model can be associated with any number of feature generators.
     Several key aspects:

       * FeatureGenerators crititcally decouple the original raw data from the features used by the models, enabling different models to use different sets of features or 
         entirely different representations of the same features.
       * FeatureGenerators become part of the model and are (where necessary) jointly trained with model. This facilitates model deployment, as every sensAI model becomes a single unit
         that can directly process raw input data, which is (usually) straightforward to supply at inference time.
       * FeatureGenerators store meta-data on the features they generate, enabling downstream components to handle them appropriately.

     The feature representation that we choose to generate can be arbitrary, but oftentimes, we will want to extract "natural" feature representations, which could, in priciple, be used by many types of models, albeit in different concrete forms.
     Sequential data can be naturally represented as an array/list, categorical data can be represented using descriptive category names, and numeric data can be represented using 
     unmodified integers and floating point numbers.

  3. **Transforming feature representations into a form that is suitable for the model at hand** (using ``DataFrameTransformers``)
     
     In the transformation stage, we address the model-specific idiosynchrasies, which may require, for example, that all features be represented as numbers (or even numbers within a limited range) or
     that all features be discrete, that no values be missing, etc.
     A ``DataFrameTransformer`` can, in principle perform an arbitary transformation from one data frame to another, but the typical use case is to apply transformations of feature representations that
     are necessary for specific types of models to work (their best).

     


     